## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [ ]:
from datasets import load_dataset, get_dataset_split_names
from torch.utils.data import DataLoader
import torch

device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

print(dataset.shape)
print(dataset["train"][0])

data_loaders = []
batch = 64

for split in dataset.keys():
    data_loaders.append(DataLoader(dataset[split], batch_size = batch, shuffle = False))

train, val, test = data_loaders

#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased", num_labels = 2)
model.to(device)

test_text = "Pippo was so silly at the beginning of the movie, he should had die since min. 1"

ttt = tokenizer(test_text, return_tensors="pt", truncation = True, padding = True) #test tokenized text

print(ttt.keys())

print()

tokens = tokenizer.convert_ids_to_tokens(ttt["input_ids"][0])
print(tokens)

print()

n_examples = 5 #len(dataset["train"])[0]
for i in range(n_examples):
    example = dataset["train"][i]["text"]

    ttt = tokenizer(example, return_tensors="pt", truncation = True, padding = True) #test tokenized text
    tokens = tokenizer.convert_ids_to_tokens(ttt["input_ids"][0])
    print(tokens)


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [ ]:
from tqdm import tqdm
import torch

def compute_features(model, tokenizer, data_loader):
  model.eval()

  features = []

  with torch.no_grad(): #ok it seems that keeping the gradient in memory makes gpu go brrrr
      for batch in tqdm(data_loader):
          tokens = tokenizer(
              batch["text"],
              padding=True,
              truncation=True,
              max_length=512,
              return_tensors="pt"
          )
          tokens = {k: v.to(device) for k, v in tokens.items()}

          outputs = model(**tokens)

          cls_embeds = outputs.last_hidden_state[:, 0, :].detach().cpu()

          features.append(cls_embeds)

          del tokens, outputs, cls_embeds
          torch.cuda.empty_cache()

  features = torch.cat(features, dim=0)
  
  return features

In [ ]:
from sklearn import svm
import numpy as np

clf = svm.SVC()

features = compute_features(model, tokenizer, train)

X = features.numpy()
y = np.array(dataset["train"]["label"])

clf.fit(X, y)

print("acc:", clf.score(X, y))

In [ ]:
features = compute_features(model, tokenizer, val)

X = features.numpy()
y = np.array(dataset["validation"]["label"])

print("acc:", clf.score(X, y))

y_pred  = clf.predict(X)
mis_idxs = np.where(y_pred != y)[0]
for i in mis_idxs[:n_examples]:
    i = int(i)
    print("Text :", dataset["validation"][i]["text"])
    print(f"Predicted  : { y_pred[i] }")

-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [ ]:
ds = dataset.map(lambda example: tokenizer(example["text"]), batched=True)
print(ds["train"][0])

#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [ ]:
from transformers import AutoModelForSequenceClassification

model_ds = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
model_ds.to(device)

#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [ ]:
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

dcwp = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred, metrics_dict):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    metrics_results = {}
    for metric in metrics_dict.keys():
        metrics_results[metric] = metrics_dict[metric](preds, labels)

    return metrics_results

metrics = {
    "acc": lambda preds, labels: (preds == labels).mean(),
    "f1": lambda preds, labels: f1_score(labels, preds, average="weighted"),
    "prec": lambda preds, labels: precision_score(labels, preds, average="weighted"),
    "rec": lambda preds, labels: recall_score(labels, preds, average="weighted")
}


training_args = TrainingArguments(
    output_dir = "./results",
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    logging_dir = "./logs",
    logging_strategy = "steps",
    logging_steps = 10,
    report_to = "tensorboard"
)

trainer = Trainer(
    model = model_ds,
    args = training_args,
    train_dataset = ds["train"],
    eval_dataset = ds["validation"],
    data_collator = dcwp,
    compute_metrics = lambda x: compute_metrics(x, metrics)
)

trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./logs

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(    
    r = 8,
    lora_alpha = 16,
    target_modules = ["q_lin", "v_lin"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = TaskType.SEQ_CLS
)

model_ds = get_peft_model(model_ds, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir = "./results",
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    logging_dir = "./logs",
    logging_strategy = "steps",
    logging_steps = 10,
    report_to = "tensorboard"
)

trainer = Trainer(
    model = model_ds,
    args = training_args,
    train_dataset = ds["train"],
    eval_dataset = ds["validation"],
    data_collator = dcwp,
    compute_metrics = lambda x: compute_metrics(x, metrics)
)

trainer.train()

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [ ]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [ ]:
# Your code here.